In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras.models import Model
from keras.layers import Input,LSTM,Dense
import numpy as np


Using TensorFlow backend.


In [0]:
batch_size = 64
epochs = 10
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
# data_path = './News.csv'
data_path = 'drive/My Drive/tabditor/수집 데이터'

In [0]:
#input은 영어 target은 프랑스어로 분리
import pandas as pd
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

data = pd.read_csv(data_path+'/title_content_split.csv')


# with open(data_path, 'r', encoding='utf-8') as f:
    # lines = f.read().split('\n')
# for line in lines[: min(num_samples, len(lines) - 1)]:
#     input_text, target_text, _ = line.split('\t')
    # target_text = '\t' + target_text + '\n'
    # input_texts.append(input_text)
    # target_texts.append(target_text)


In [0]:
print(data.shape)
data.head()

(8916, 6)


Unnamed: 0  ...  between
0           0  ...      \n대
1           1  ...      \n중
2           2  ...      \n2
3           3  ...      \n새
4           4  ...      \n반

[5 rows x 6 columns]

In [0]:
data['title'].str.contains('불가능성은').sum()

7

In [0]:
data['title'].str.contains('불가능은').sum()

1

In [0]:
import os 

os.sys.path.append('drive/My Drive/tabditor/소스코드/기능4. 제목 추출/')

from krt_p3_modified import *

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8916 entries, 0 to 8915
Data columns (total 6 columns):
Unnamed: 0    8916 non-null int64
jss_id        8916 non-null int64
nb_pat        8916 non-null int64
title         8916 non-null object
content       8914 non-null object
between       8916 non-null object
dtypes: int64(3), object(3)
memory usage: 418.1+ KB


In [0]:
data['content'][:100]

0     대학 3학년 때 교내 배구 서포터즈 활동을 했습니다. 타과생들에겐 생소할 수 있는 ...
1     중학교 정규수업을 들으며 스포츠를 처음 접했으며 모든 종목 수행평가에서 만점을 받으...
2     2년 전 카페의 관리자로 근무하며, 과발주로 인해 매장의 매출 손실을 막은 경험이 ...
3     새로운 지식을 수용하는 데 탁월한 학습능력을 갖추고 있습니다.\n대학 2학년 때 가...
4     반면 현재에 집중하기 보다 앞선 걱정을 많이 하는 경향이 있습니다. 특히 시험을 준...
                            ...                        
95    저는 CJ그룹이 안정적인 IT시스템 운영을 위해 주요 계열사의 IT시스템을 모두 클...
96    2학년 2학기에 '소프트웨어설계'과목을 수강하여 UML을 이용한 소프트웨어 시스템 ...
97    부모님의 사정으로 잦은 전학을 경험하며 매번 새로운 상황에 적응해야 했습니다. 어색...
98    최근 온라인 및 모바일 거래가 활성화되면서 오프라인 은행 지점이 급격히 줄고 있습니...
99    두 번의 봉사활동을 통해 '상대방의 입장에서 생각하는 것의 중요성'을 깨달은 적이 ...
Name: content, Length: 100, dtype: object

In [0]:
# 결측치 제거
data = data.dropna(axis=0)

In [0]:
data.loc[data['content'].isnull()] 

Empty DataFrame
Columns: [Unnamed: 0, jss_id, nb_pat, title, content, between]
Index: []

In [0]:
import re 
pat = re.compile('[^가-힣a-zA-Z1-9 ]')

# data['content'].apply(lambda x :romanize(pat.sub('',x)).lower())

In [0]:
input_texts = data['content'].apply(lambda x : romanize(pat.sub('',x)).lower())
target_texts = data['title'].apply(lambda x: romanize('\t'+ pat.sub('',x)).lower())

# input_characters = set()
# target_characters = set()

for sentence in input_texts:    
    for char in sentence:
        if char not in input_characters:
            input_characters.add(char)
for sentence in target_texts:
    for char in sentence:
        if char not in target_characters:
            target_characters.add(char)


In [0]:
input_texts

0       tay.hak 3haknyen ttay kyo.nay pay.kwu se.pho.t...
1       cwunghak.kyo cengkyu.swu.ep.ul tul.u.mye su.ph...
2       2nyen cen kha.phey.uy kwanli.ca.lo kunmwu.ha.m...
3       say.lo.wun ci.sik.ul swu.yongha.nun tey thak.w...
4       panmyen hyen.cay.ey cipcwungha.ki po.ta aphsen...
                              ...                        
8911    chang.ep ci.wen kongmo.cen.ey chamka.hayss.sup...
8912     senpay.tul.uy chwu.chen.ul pat.a 13nyento ton...
8913     no.lyek.ha.nun kes i.sang.u.lo cwung.yo.han k...
8914    cenkongkwa.mok cwung so.pi.ca hayngtongpwunsek...
8915    68nyen.uy centhongkwa choy.ko.cham sangphyo.lu...
Name: content, Length: 8914, dtype: object

In [0]:
news_data = pd.read_csv('/content/drive/My Drive/tabditor/소스코드/기능4. 제목 추출/news_headline_text.csv')

news_data.head()

Unnamed: 0  ...                                               text
0           0  ...  자유한국당은 13일 북한 주민 송환과 관련한 논란에 화력을 집중하며 이 문제를 낱낱...
1           1  ...  선임의 구타와 가혹행위로 스스로 목숨을 끊은 장병 6명이 뒤늦게 '순직' 판정을 받...
2           2  ...  북한이 김일성 주석의 베트남 방문 55주년을 계기로 양국간 친선 우호를 재차 강조했...
3           3  ...  마크 밀리 미국 합참의장이 13일 한국을 방문한다.취임 후 처음으로 일본에 이어 한...
4           4  ...  더불어민주당이 내년 총선에서 '청년신도시 조성'을 핵심공약 중 하나로 내세우는 방안...

[5 rows x 3 columns]

In [0]:

pat2 = re.compile('[一-龥]')
input_texts2 = [pat2.sub('',romanize(i).lower()) for i in news_data['text']]
target_texts2 = [pat2.sub('',romanize(i).lower()) for i in news_data['summary'].apply(lambda x: '\t'+ x + '\n')]
pat = re.compile('[^一-龥]')

input_characters = set()
target_characters = set()

for sentence in input_texts2:    
    for char in sentence:
        if pat.search(char):
            if char not in input_characters:
                input_characters.add(char)
            
for sentence in target_texts2:
    for char in sentence:
        if pat.search(char):
            if char not in target_characters:
                target_characters.add(char)

In [0]:
target_texts[:10]

0    \tkyo.nay su.pho.chu kyengki hwal.senghwa.lul ...
1             \tsu.pho.chu.lul hyanghan o.layn kwansim
2    \tmay.chwul sonsil.ul mak.ki wi.han kwanli.ca....
3    \tthak.wel.han ci.sik swu.yong nunglyek.kwa un...
4    \tchey.kyey.cek.in kyey.hoyk.ul thonghan kekce...
5            \tso.thongkwa kongkam.un mansa.hyengthong
6              \ttan.chey.uy cay.ceng.ul kay.sen.ha.ta
7                            \tinsayng.uy cen.hwan.cem
8           \tto.si.konghak.ey.se san.epkonghak.kka.ci
9          \thankwuk tho.pak.i.uy hay.oy.inthen to.cen
Name: title, dtype: object

In [0]:
print(data.shape)
data.head() 

(8914, 6)


Unnamed: 0  ...  between
0           0  ...      \n대
1           1  ...      \n중
2           2  ...      \n2
3           3  ...      \n새
4           4  ...      \n반

[5 rows x 6 columns]

In [0]:
input_characters

{' ',
 '!',
 '"',
 '#',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 ']',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '}',
 '~',
 '\xa0',
 '\xad',
 '±',
 '·',
 'α',
 '‘',
 '’',
 '…',
 '※',
 'ℓ',
 '⅓',
 '↑',
 '→',
 '↓',
 '∼',
 '■',
 '▲',
 '▶',
 '▼',
 '◆',
 '◇',
 '○',
 '☎',
 '♡',
 '♬',
 'か',
 'な',
 '㎍',
 '㎎',
 '㎏',
 '㎘',
 '㎝',
 '㎞',
 '㎡',
 '㎥',
 '㏄',
 '奈',
 '論',
 '不',
 '兩',
 '立',
 'ｍ',
 '｜'}

In [0]:
print(len(input_characters))
print(len(target_characters))

105
61


In [0]:
#고유 문자 개수 및 단어 길이가 가장 긴 값을 변수에 지정.
import numpy as np
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = int(np.mean([len(txt) for txt in input_texts]))
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 8914
Number of unique input tokens: 105
Number of unique output tokens: 61
Max sequence length for inputs: 1436
Max sequence length for outputs: 179


In [0]:
# 딕셔너리에 인덱스로 값을 변환해서 저장. 
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# 

In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # if t>=1436: 이것때매 실행이 안됬네? 
    #     break   
    for t, char in enumerate(input_text):
        if t>=1436:
            break
        # try:   
        encoder_input_data[i, t, input_token_index[char]] = 1.
        # print(char)
        # except:
        #     pass
    # try:
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    # except:
    #     pass
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        
        # try:
        decoder_input_data[i, t, target_token_index[char]] = 1.
        # except:
        #     pass
        
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            # try:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
            # except:
            #     pass
    if t>=1436:
        break
    # try:
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

    # except:
    #     pass

In [0]:
encoder_input_data[0][3]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [0]:
encoder_input_data.shape

(8914, 1436, 105)

In [0]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [0]:
#decoder_inputs 의 인풋모듈은 무엇이며 이걸 lstm에 넘겨주면 어떻게 받는건가?
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [0]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model.load_weights('drive/My Drive/tabditor/소스코드/기능4. 제목 추출/s2s_jasoseo.h5')

In [0]:
from keras.callbacks import ModelCheckpoint 
cb = ModelCheckpoint('drive/My Drive/tabditor/소스코드/기능4. 제목 추출/s2s_jasoseo.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [0]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=200,
          callbacks=[cb],
          validation_split=0.2)
# Save model


In [0]:
model.save('drive/My Drive/summary_folder/s2s_news.h5')

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
target_text

'\tsey.kyey.ka saymnay.nun ki.ep saymphyo'

In [0]:
for seq_index in range(50):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', hangulize(input_texts[seq_index],toEncoding='utf-8'))
    print('Decoded sentence:', hangulize(decoded_sentence,toEncoding='utf-8'))
    print('answer sentence:', hangulize(data['title'][seq_index],toEncoding='utf-8'))

-
Input sentence: 대학 3학년 때 교내 배구 서포터즈 활동을 했습니다 타과생들에겐 생소할 수 있는 대학 간 스포츠 경기가 활성화 될 수 있도록 학교 내 유동인구가 많은 지역에서 간식을 나눠주며 경기일정을 홍보하였습니다 경기장을 방문하는 관객을 대상으로 다양한 행사를 실시하여 관객유치를 위해 노력하였습니다 그 결과 연속 3경기 만원 관중을 달성하는 성과를 거두었습니다아쉬운점 1
Decoded sentence: 경쟁                                                                                                                                                                          
answer sentence: "교내 스포츠 경기 활성화를 위한 노력"


-
Input sentence: 중학교 정규수업을 들으며 스포츠를 처음 접했으며 모든 종목 수행평가에서 만점을 받으며 스포츠관련 학과에 진학하는 목표를 세웠습니다 대학 입학 후 테니스 동아리 배구동아리를 통해 대회에 참가하였고 플로어볼과 크리켓 같은 생소한 스포츠를 접하여 스포츠에 대한 다양한 지식을 얻을 수 있었습니다좋은점 1 글자수 49자74byt어
Decoded sentence: 가장 큰 실패와 성공을 가져온 호주 워킹홀리데이                                                                                                           
answer sentence: "스포츠를 향한 오랜 관심"


-
Input sentence: 2년 전 카페의 관리자로 근무하며 과발주로 인해 매장의 매출 손실을 막은 경험이 있습니다 승진한 지 얼마 되지 않은 수퍼바이저의 업무 미숙으로 인해 이틀에 1개 정도 팔리는 샐러드 상품 3개가 매장에 입고된 적이 있었습니다 유통기한을 3일로 아주 짧아 팔리